In [ ]:
!pip install pyspark -q
!pip install findspark -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# Création d'une session Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Final Project").getOrCreate()

## Téléchargement des données

Source des données : https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2/about_data

In [ ]:
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD

--2024-04-13 19:01:05--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.199, 52.206.68.26, 52.206.140.205
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [          <=>       ]   1.77G  3.15MB/s    in 10m 36s 

2024-04-13 19:11:43 (2.84 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [1898215669]



In [ ]:
!ls

'rows.csv?accessType=DOWNLOAD'	 sample_data


In [ ]:
!mv rows.csv?accessType=DOWNLOAD reported-crimes.csv

## Ingestion des données

In [ ]:
# Lecture du fichier
df = spark.read.csv(
    'reported-crimes.csv',
    header = True
)

df.show(5)

+--------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
| 5741943|   HN549294|08/25/2007 09:22:...|  074XX N ROGERS AVE|0560|     ASSAULT|              SIMPLE|               OTHER| false|   false|2422|   

In [ ]:
df.count()

8037361

In [ ]:
type(df)

pyspark.sql.dataframe.DataFrame

In [ ]:
# Type des variables
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



## Manipulation des données

In [ ]:
from pyspark.sql.functions import to_timestamp, col

In [ ]:
df = df.withColumn(
    'Date',
    to_timestamp(col('Date'), 'MM/dd/yyyy hh:mm:ss a')
)

df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [ ]:
df.show(5)

+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
| 5741943|   HN549294|2007-08-25 09:22:18|  074XX N ROGERS AVE|0560|     ASSAULT|              SIMPLE|               OTHER| false|   false|2422|     02

In [ ]:
df.columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [ ]:
df.select(col('Description')).show(5)

+--------------------+
|         Description|
+--------------------+
|              SIMPLE|
| FIRST DEGREE MURDER|
| FIRST DEGREE MURDER|
|      UNLAWFUL ENTRY|
|AGGRAVATED P.O. -...|
+--------------------+
only showing top 5 rows



In [ ]:
df.select('Date', 'Description', 'Arrest').show(5)

+-------------------+--------------------+------+
|               Date|         Description|Arrest|
+-------------------+--------------------+------+
|2007-08-25 09:22:18|              SIMPLE| false|
|2021-05-24 15:06:00| FIRST DEGREE MURDER|  true|
|2021-06-26 09:24:00| FIRST DEGREE MURDER|  true|
|2023-11-09 07:30:00|      UNLAWFUL ENTRY| false|
|2023-11-12 07:59:00|AGGRAVATED P.O. -...|  true|
+-------------------+--------------------+------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import lit

In [ ]:
df.withColumn('One', lit(1))

DataFrame[ID: string, Case Number: string, Date: timestamp, Block: string, IUCR: string, Primary Type: string, Description: string, Location Description: string, Arrest: string, Domestic: string, Beat: string, District: string, Ward: string, Community Area: string, FBI Code: string, X Coordinate: string, Y Coordinate: string, Year: string, Updated On: string, Latitude: string, Longitude: string, Location: string, One: int]

In [ ]:
df.columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [ ]:
# Crimes signalés le 25-12-2023
one_day = df.filter(col('Date') == lit('2023-12-25'))
one_day

DataFrame[ID: string, Case Number: string, Date: timestamp, Block: string, IUCR: string, Primary Type: string, Description: string, Location Description: string, Arrest: string, Domestic: string, Beat: string, District: string, Ward: string, Community Area: string, FBI Code: string, X Coordinate: string, Y Coordinate: string, Year: string, Updated On: string, Latitude: string, Longitude: string, Location: string]

In [ ]:
one_day.count()

24

In [ ]:
one_day.show()

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|13318703|   JG554228|2023-12-25 00:00:00|     035XX W 58TH ST|5001|       OTHER OFFENSE|OTHER CRIME INVOL...|           APARTM

In [ ]:
one_day.distinct().count()

24

## Analyse des données

In [ ]:
df.groupBy('Primary Type').count().show()

+--------------------+-------+
|        Primary Type|  count|
+--------------------+-------+
|OFFENSE INVOLVING...|  57608|
|CRIMINAL SEXUAL A...|   8483|
|            STALKING|   5284|
|PUBLIC PEACE VIOL...|  53166|
|           OBSCENITY|    865|
|               ARSON|  13748|
|   DOMESTIC VIOLENCE|      1|
|            GAMBLING|  14634|
|   CRIMINAL TRESPASS| 218787|
|             ASSAULT| 529230|
|LIQUOR LAW VIOLATION|  15085|
| MOTOR VEHICLE THEFT| 401467|
|               THEFT|1697834|
|             BATTERY|1465855|
|             ROBBERY| 302969|
|            HOMICIDE|  13142|
|           RITUALISM|     24|
|    PUBLIC INDECENCY|    200|
| CRIM SEXUAL ASSAULT|  27503|
|   HUMAN TRAFFICKING|    105|
+--------------------+-------+
only showing top 20 rows



In [ ]:
df.select("Arrest").distinct().show()

+------+
|Arrest|
+------+
| false|
|  true|
+------+



In [ ]:
df.filter(col("Arrest") == "true").count() / df.select("Arrest").count()

0.25712481995022995

In [ ]:
n_location = df.groupBy("Location Description").count()
n_location.show()

+--------------------+------+
|Location Description| count|
+--------------------+------+
|   RAILROAD PROPERTY|    15|
|SCHOOL - PRIVATE ...|  1040|
|AIRPORT TERMINAL ...|  2489|
|VEHICLE - COMMERCIAL|   863|
|EXPRESSWAY EMBANK...|     1|
|RESIDENCE - YARD ...| 11565|
|POLICE FACILITY/V...| 18566|
|               MOTEL|     7|
|CHA PARKING LOT /...|  1783|
|            SIDEWALK|741740|
|AIRPORT TERMINAL ...|   131|
|PUBLIC GRAMMAR SC...|     2|
|OTHER RAILROAD PR...|   477|
|            CAR WASH|  3450|
|   TRUCKING TERMINAL|     1|
|    AIRPORT/AIRCRAFT| 16284|
|            HOSPITAL|    14|
|    FEDERAL BUILDING|  1003|
|MEDICAL/DENTAL OF...|  7430|
|             TRAILER|     4|
+--------------------+------+
only showing top 20 rows



In [ ]:
n_location.orderBy('count', ascending=False).show()

+--------------------+-------+
|Location Description|  count|
+--------------------+-------+
|              STREET|2099565|
|           RESIDENCE|1336324|
|           APARTMENT| 926886|
|            SIDEWALK| 741740|
|               OTHER| 270006|
|PARKING LOT/GARAG...| 202963|
|               ALLEY| 178758|
|  SMALL RETAIL STORE| 154693|
|SCHOOL, PUBLIC, B...| 146373|
|    RESIDENCE-GARAGE| 135523|
|          RESTAURANT| 132621|
|VEHICLE NON-COMME...| 127595|
|RESIDENCE PORCH/H...| 124190|
|    DEPARTMENT STORE| 102897|
|  GROCERY FOOD STORE| 101105|
|         GAS STATION|  89470|
|RESIDENTIAL YARD ...|  75148|
|COMMERCIAL / BUSI...|  63920|
|       PARK PROPERTY|  60445|
|CHA PARKING LOT/G...|  56101|
+--------------------+-------+
only showing top 20 rows



In [ ]:
df.select('Primary Type').show(30, truncate=False)

+--------------------------+
|Primary Type              |
+--------------------------+
|ASSAULT                   |
|HOMICIDE                  |
|HOMICIDE                  |
|BURGLARY                  |
|BATTERY                   |
|THEFT                     |
|CRIMINAL DAMAGE           |
|DECEPTIVE PRACTICE        |
|CRIMINAL SEXUAL ASSAULT   |
|THEFT                     |
|CRIMINAL DAMAGE           |
|THEFT                     |
|DECEPTIVE PRACTICE        |
|THEFT                     |
|THEFT                     |
|DECEPTIVE PRACTICE        |
|OFFENSE INVOLVING CHILDREN|
|MOTOR VEHICLE THEFT       |
|THEFT                     |
|DECEPTIVE PRACTICE        |
|MOTOR VEHICLE THEFT       |
|MOTOR VEHICLE THEFT       |
|ROBBERY                   |
|ASSAULT                   |
|ROBBERY                   |
|CRIMINAL SEXUAL ASSAULT   |
|ROBBERY                   |
|OFFENSE INVOLVING CHILDREN|
|OFFENSE INVOLVING CHILDREN|
|OFFENSE INVOLVING CHILDREN|
+--------------------------+
only showing t

In [ ]:
n_crime_type = df.select('Primary Type').distinct().count()
n_crime_type

36

In [ ]:
df.select('Primary Type').distinct().show(n_crime_type, truncate=False)

+---------------------------------+
|Primary Type                     |
+---------------------------------+
|OFFENSE INVOLVING CHILDREN       |
|CRIMINAL SEXUAL ASSAULT          |
|STALKING                         |
|PUBLIC PEACE VIOLATION           |
|OBSCENITY                        |
|ARSON                            |
|DOMESTIC VIOLENCE                |
|GAMBLING                         |
|CRIMINAL TRESPASS                |
|ASSAULT                          |
|LIQUOR LAW VIOLATION             |
|MOTOR VEHICLE THEFT              |
|THEFT                            |
|BATTERY                          |
|ROBBERY                          |
|HOMICIDE                         |
|RITUALISM                        |
|PUBLIC INDECENCY                 |
|CRIM SEXUAL ASSAULT              |
|HUMAN TRAFFICKING                |
|INTIMIDATION                     |
|PROSTITUTION                     |
|DECEPTIVE PRACTICE               |
|CONCEALED CARRY LICENSE VIOLATION|
|SEX OFFENSE                

In [ ]:
# Dataframe des activités non-criminelles (à revoir)
nc = df.filter(
    (col('Primary Type') == 'NON - CRIMINAL') | (col('Primary Type') == 'NON-CRIMINAL') | (col('Primary Type') == 'NON-CRIMINAL (SUBJECT SPECIFIED)')
)

nc.show()

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|13280353|   JG507932|2023-11-17 13:17:00|014XX S BLUE ISLA...|1481|        NON-CRIMINAL|CONCEALED CARRY L...|POLICE FACILITY /

In [ ]:
nc.count()

231

In [ ]:
nc.groupBy('Description').count().orderBy('count', ascending=False).show()

+--------------------+-----+
|         Description|count|
+--------------------+-----+
|       LOST PASSPORT|  111|
|   FOID - REVOCATION|   76|
|CONCEALED CARRY L...|   20|
|NOTIFICATION OF C...|    9|
|NOTIFICATION OF S...|    8|
|      FOUND PASSPORT|    4|
|GUN OFFENDER NOTI...|    3|
+--------------------+-----+



In [ ]:
from pyspark.sql import functions
dir(functions)

['Any',
 'ArrayType',
 'Callable',
 'Column',
 'DataFrame',
 'DataType',
 'Dict',
 'Iterable',
 'JVMView',
 'List',
 'Optional',
 'PandasUDFType',
 'PySparkTypeError',
 'PySparkValueError',
 'SparkContext',
 'StringType',
 'StructType',
 'TYPE_CHECKING',
 'Tuple',
 'Type',
 'Union',
 'UserDefinedFunction',
 'UserDefinedTableFunction',
 'ValuesView',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_create_column_from_literal',
 '_create_lambda',
 '_create_py_udf',
 '_create_py_udtf',
 '_from_numpy_type',
 '_get_jvm_function',
 '_get_lambda_parameters',
 '_invoke_binary_math_function',
 '_invoke_function',
 '_invoke_function_over_columns',
 '_invoke_function_over_seq_of_columns',
 '_invoke_higher_order_function',
 '_options_to_str',
 '_test',
 '_to_java_column',
 '_to_seq',
 '_unresolved_named_lambda_variable',
 'abs',
 'acos',
 'acosh',
 'add_months',
 'aes_decrypt',
 'aes_encrypt',
 'aggregate',
 'any_value',
 'approxC

In [ ]:
len(dir(functions))

484